# Libraries

In [11]:
import warnings
warnings.filterwarnings("ignore")
from torchvision.io.video import read_video
from torchvision.models.video import r3d_18,R3D_18_Weights,s3d,S3D_Weights, mc3_18, MC3_18_Weights, r2plus1d_18, R2Plus1D_18_Weights
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision.io import read_video
import torch.optim as optim
import matplotlib.pyplot as plt
import torch
import random
from collections import defaultdict
import numpy as np
import time
import shutil
import pandas as pd
import tqdm
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader
import os

# Models_params

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 1

EPOCHS = 100

FEATURE_EXTRACT = True

USE_WEIGHTS = True

MODELS_NAMES = ['MViT', 'ResNet', 's3d']

TRAIN_SIZE = 0.8


In [9]:
train_dir = './train'
validation_dir = './validation'

for  i in [train_dir, validation_dir]:
    try:
        os.mkdir(i)
    except FileExistsError as ex:
        pass

In [10]:
videos_filename = os.listdir('./data/videos')
random.shuffle(videos_filename)
videos_number = len(videos_filename)

for i, vid in enumerate(videos_filename):
    if i < round(videos_number * TRAIN_SIZE):
        shutil.move('./data/videos/'+vid,train_dir)
    else:
        shutil.move('./data/videos/'+vid,validation_dir)

FileNotFoundError: [Errno 2] No such file or directory: './data/videos'

In [ ]:
def create_csv_dataset(path):
    df = pd.DataFrame(columns=['vid','target'])
    vid_names = os.listdir(path)
    for vid_name in tqdm(vid_names):
        name, _ = vid_name.split('%')
        _,persentage = name.split(', ')
        df.loc[len(df)] = [vid_name, float(persentage)]

    if "train" in path:
        df.to_csv(f'./train_data.csv')
    else:
        df.to_csv(f'./val_data.csv')
    return df

In [ ]:
def initialize_model(model_name, feature_extract, use_weights):
    
    """
    Инициализация модели, которая будет использована для обучения.
    
    Args:
        model_name (_str_): Название используемой модели из документации.
        feature_extract (_bool_): _description_
        use_weights (_bool_): Парамет, указывающий, будем ли мы использовать предобученные веса для данной модели.
    """

    if 'r3d_18' in model_name:
        if use_weights:
            weights = R3D_18_Weights.KINETICS400_V1
            model = r3d_18(weights=weights)    
        else:
            model = r3d_18()

    if 'mc3_18' in model_name:
        if use_weights:
            weights = MC3_18_Weights.KINETICS400_V1
            model = mc3_18(weights=weights)
        else:
            model = mc3_18()

    if 'r2plus1d_18' in model_name:
        if use_weights:
            weights = R2Plus1D_18_Weights.KINETICS400_V1
            model = r2plus1d_18(weights=weights)
        else:
            model = r2plus1d_18()

    if 's3d' in model_name:
        if use_weights:
            weights = S3D_Weights.KINETICS400_V1
            model = s3d(weights=weights)
        else:
            model = s3d()

    model.fc = nn.Linear(in_features=512, out_features=1, bias=True)

    if feature_extract:
        for name, param in model.named_parameters():
            if 'fc' not in name:
                param.requires_grad = False

    return model

In [ ]:

class VideoRegressionDataset(Dataset):
    
    def __init__(self, videos_dir, transform=None):
        """
        Arguments:
            videos_dir (string): Path to the directory containing video files.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        print("Creating dataset...")
        self.value_to_frame = create_csv_dataset(videos_dir)
        print("Dataset created!")
        self.videos_dir = videos_dir
        self.transformer = transform

    def __len__(self):
        return len(self.value_to_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        video_name = os.path.join(self.videos_dir, self.value_to_frame.iloc[idx, 0])
        video, _, _ = read_video(video_name, pts_unit="sec")

        frames = video.permute(0, 3, 244, 244)  # Подставить нужные размеры [T, C, H, W]
        targets = float(self.value_to_frame.iloc[idx, 1:])
        sample = {'video': frames, 'target': targets}

        if self.transformer:
            sample = self.transformer(sample)

        return sample

In [ ]:
train_dataloader = DataLoader(VideoRegressionDataset(
    videos_dir='./train',
    transform=transforms.Compose([
        Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

validation_dataloader = DataLoader(VideoRegressionDataset(
    videos_dir='./validation',
    transform=transforms.Compose([
        Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
def plot_learning_curves(history):
    '''
    Функция для построения графиков лоса и точности.

    :param history: (dict)
        accuracy и loss на обучении и валидации
    '''
    fig = plt.figure(figsize=(20, 7))

    plt.subplot(1,2,1)
    plt.title('Лосс', fontsize=15)
    plt.plot(history['loss']['train'], label='train')
    plt.plot(history['loss']['val'], label='val')
    plt.ylabel('лосс', fontsize=15)
    plt.xlabel('эпоха', fontsize=15)
    plt.legend()

In [12]:
def train(
    model,
    model_params,
    criterion,
    optimizer,
    train_dataloader,
    val_dataloader,
    num_epochs=15
):
   

    history = defaultdict(lambda: defaultdict(list))
    valid_loss_min = np.inf
    
    current_dir = os.getcwd()
    model_save_dir = f'./models/{model_params[0]}'
    try:
        os.mkdir(model_save_dir)
    except FileExistsError as ex:
        pass

    for epoch in range(num_epochs):
        train_loss = 0
        val_loss = 0

        start_time = time.time()

        # Set the model to training mode
        model.train(True)
        model.to(DEVICE)

        # Training loop
        print('Training')
        for batch in tqdm(train_dataloader):
            # Move batch to the device
            X_batch = batch['video'].to(DEVICE)
            y_batch = batch['target'].to(DEVICE)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_dataloader)
        history['loss']['train'].append(train_loss)

        # Set the model to evaluation mode
        model.eval()
        print('Validation')
        with torch.no_grad():
            # Validation loop
            for batch in tqdm(val_dataloader):
                # Move batch to the device
                X_batch = batch['video'].to(DEVICE)
                y_batch = batch['target'].to(DEVICE)

                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_dataloader)
        history['loss']['val'].append(val_loss)

        clear_output()


        # Print results after each epoch
        print(f'Model: {model_params[0]}. LR: {model_params[1]}. Momentum: {model_params[2]}')
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss: \t{:.6f}".format(train_loss))
        print("  validation loss: \t{:.6f}".format(val_loss))
        
        if val_loss < valid_loss_min:
            print(f'Detected network improvement on epoch {epoch},saving the current model')
            valid_loss_min = val_loss
            torch.save(model.cpu(), model_save_dir + f'\\{model_params[0]}_lr{model_params[1]}_mom{model_params[2]}.pth')
        
        plot_learning_curves(history)
    
    model.cpu()   
    return model, history

In [ ]:
def prepare_optimizer_and_criterion(model, learning_rate, moment):
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad:
            params_to_update.append(param)
            print("\t", name + " updating")
    optimizer_ft = torch.optim.SGD(
        params_to_update,
        lr=learning_rate,
        momentum=moment,
        weight_decay=0.05
    )
    criterion = nn.MSELoss()
    return optimizer_ft, criterion


In [ ]:
current_directory = os.getcwd()

try:
    os.mkdir(current_directory + '\\models')
except FileExistsError as ex:
    pass

for model_name in MODELS_NAMES:
    for learning_rate in [0.0003]:
        for moment in [0.9]:
            model, input_size = initialize_model(model_name, FEATURE_EXTRACT, USE_WEIGHTS)  
            model = model.to(DEVICE)
            optimizer, criterion = prepare_optimizer_and_criterion(model, learning_rate, moment)
            model_params = [model_name, learning_rate, moment]
            model, history = train(
                model,
                model_params,
                criterion,
                optimizer,
                train_dataloader,
                validation_dataloader,
                EPOCHS
            )
            time.sleep(5)
            torch.cuda.empty_cache()